# Análise da taxa de câmbio do DOLAR ao longo do tempo

Nesse estudo, será analisado como a taxa de câmbio entre o dolar e o real se comportou ao longo do tempo. Para isso, serão analisadas tanto a taxa de câmbio do dolar comercial quanto do dolar paralelo. O interessante de avaliar a taxa do mercado paralelo é verificar se, em momentos onde a taxa comercial é controlada, a taxa do mercado paralelo segue de maneira mais real a taxa verdadeira. 

O valor da taxa de câmbio será ajustado tanto pela inflação brasileira quanto pela inflação americana. 

Várias informações podem ser obtidos no [IPEAData](http://ipeadata.gov.br) e será utilizada a biblioteca [`ipeadatapy`](https://github.com/luanborelli/ipeadatapy/) para acesso a esses dados.

In [ ]:
import ipeadatapy as idpy

Serão utilizadas as seguintes séries mensais:

- Taxa de câmbio - `R$ / US$` - comercial - venda - fim período
- Taxa de câmbio - `R$ / US$` - paralelo - venda - fim período

In [ ]:
idpy.list_series(name='Taxa de câmbio - R$ / US$ - comercial - venda - fim período')

In [ ]:
idpy.list_series(name='Taxa de câmbio - R$ / US$ - paralelo - venda - fim período')

In [ ]:
idpy.describe('BM12_ERVF12')

In [ ]:
idpy.describe('BM_ERVF')

In [ ]:
idpy.describe('GM12_BLACKF12')

Como estamos interessado na análise mensal, serão utilizadas as séries `BM12_ERVF12` e `GM12_BLACKF12`.

In [ ]:
dolar_comercial = idpy.timeseries("BM12_ERVF12")
dolar_comercial

Podemos observar que a série do dolar comercial inicia-se em janeiro/1953, indo até hoje.

In [ ]:
dolar_paralelo = idpy.timeseries('GM12_BLACKF12')
dolar_paralelo

Podemos observar que a série do dolar paralelo inicia-se em setembro/1947, encerrando em fevereiro/2016.

Faremos um gráfico das duas séries, em escala logarítmica.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure()
plt.plot(dolar_comercial['VALUE (R$)'], label = 'comercial')
plt.plot(dolar_paralelo['VALUE (R$)'], label = 'paralelo')
plt.legend()
plt.yscale('log')
plt.show()

Observamos que há um pico no gráfico do dolar paralelo, próximo de 1970, que precisa ser verificado.

In [ ]:
df = dolar_paralelo.loc['1965-01-01':'1970-01-01']
print(df['VALUE (R$)'].describe())


Podemos observar que, reduzindo-se o intervalo da série para o intervalo entre 1965 e 1970, identificamos que o valor máximo é muito discrepante em relação a média.

In [ ]:
df[['VALUE (R$)']].idxmax()

In [ ]:
df.loc['1968-05-01':'1969-05-01']

Podemos observar que, aparentemente, há um erro no expoente. Deveria ser E**-12 quando é E**-9. As mantissas estão coerentes. Vamos atualizar esse valor e fazer novamente o gráfico para conferir.

In [ ]:
df.loc['1968-11-01','VALUE (R$)']

In [ ]:

dolar_paralelo.loc['1968-11-01','VALUE (R$)'] = dolar_paralelo.loc['1968-11-01','VALUE (R$)'] /1000
dolar_paralelo.loc['1968-11-01','VALUE (R$)']

In [ ]:
fig = plt.figure()
plt.plot(dolar_comercial['VALUE (R$)'], label = 'comercial')
plt.plot(dolar_paralelo['VALUE (R$)'], label = 'paralelo')
plt.legend()
plt.yscale('log')
plt.show()

Verificamos que o pico sumiu.

Para o ajuste da taxa de câmbio pela inflação brasileira, o ideal seria a utilização do IPCA, que é o índice utilizado atualmente pelo Banco Central para acompnhar a inflação, mas esse índice foi criado apenas na década de 80, e não cobre o período total das taxas de câmbio. O IGP-DI é mais antigo, servindo melhor aos propósitos da análise em uma escala de tempo maior. De qualquer maneira, será realizadas análises com os 2 índices de inflação.

Serão utilizadas as seguintes séries mensais:

- Preços - IGP-DI - geral - índice (ago. 1994 = 100)
- Preços - IPCA - geral - índice (dez. 1993 = 100)

In [ ]:
idpy.list_series(name='Preços - IGP-DI - geral - índice (ago. 1994 = 100)')

In [ ]:
idpy.list_series(name='Preços - IPCA - geral - índice (dez. 1993 = 100)')

In [ ]:
idpy.describe('IGP_IGP')

In [ ]:

idpy.describe('IGP12_IGPDI12')

In [ ]:
idpy.describe('PRECOS12_IPCA12')

Como estamos interessado na análise mensal, serão utilizadas as séries `IGP12_IGPDI12` e `PRECOS12_IPCA12`.

In [ ]:
igp_di = idpy.timeseries('IGP12_IGPDI12')
igp_di

Podemos observar que a série do IGP-DI inicia-se em janeiro/1944, indo até hoje.

In [ ]:
ipca = idpy.timeseries('PRECOS12_IPCA12')
ipca

Podemos observar que a série do IPCA inicia-se em dezembro/1979, indo até hoje.

Para se ajustar os valores do dolar pela inflação, é necessário uma data de referência. Inicialmente, vamos considerar a data mais recente do índice de inflação.

In [ ]:
last_igp = igp_di['VALUE (-)'].iat[-1]
print(last_igp)

In [ ]:
last_ipca = ipca['VALUE (-)'].iat[-1]
print(last_ipca)

Vamos criar um novo `dataframe` com as colunas do dolar ajustadas pelos índicies de inflação.

In [ ]:
from pandas.core.frame import DataFrame

In [ ]:
dolar = DataFrame()
dolar['comercial'] = dolar_comercial['VALUE (R$)']
dolar['paralelo'] = dolar_paralelo['VALUE (R$)']
dolar['igp'] = igp_di['VALUE (-)']
dolar['ipca'] = ipca['VALUE (-)']
dolar['comercial_igp_atual'] = dolar['comercial'] * last_igp/dolar['igp']
dolar['paralelo_igp_atual'] = dolar['paralelo'] * last_igp/dolar['igp']
dolar['comercial_ipca_atual'] = dolar['comercial'] * last_ipca/dolar['ipca']
dolar['paralelo_ipca_atual'] = dolar['paralelo'] * last_ipca/dolar['ipca']
dolar

In [ ]:
fig = plt.figure()
plt.plot(dolar['comercial_igp_atual'], label = 'comercial (IGP-DI)')
plt.plot(dolar['paralelo_igp_atual'], label = 'paralelo (IGP-DI)')
plt.plot(dolar['comercial_ipca_atual'], label = 'comercial (IPCA)')
plt.plot(dolar['paralelo_ipca_atual'], label = 'paralelo (IPCA)')
plt.legend()
plt.show()

Com relação ao índice de inflação americana, embora o IPEAData tenha os dados do CPI, essa série não está disponível na API. 

Os dados da inflação americana (Consumer Price Index for All Urban Consumers: All Items in U.S. City Average) estão disóníveis no [Federal Reserve Bank of St. Louis](https://fred.stlouisfed.org/) e será utilizada a biblioteca `pandas_datareaader` para acesso a esses dados. 

In [ ]:
import pandas_datareader.data as web
import datetime

In [ ]:
start = datetime.datetime(1940, 1, 1)
cpi = web.DataReader('CPIAUCSL', 'fred', start )
cpi

Podemos observar que a série do CPI inicia-se em janeiro/1947, indo até hoje.

In [ ]:
last_cpi = cpi['CPIAUCSL'].iat[-1]
print(last_cpi)

In [ ]:
dolar['cpi'] = cpi

In [ ]:
dolar['comercial_igp_cpi_atual'] = dolar['comercial_igp_atual']*dolar['cpi']/last_cpi
dolar['paralelo_igp_cpi_atual'] = dolar['paralelo_igp_atual']*dolar['cpi']/last_cpi
dolar['comercial_ipca_cpi_atual'] = dolar['comercial_ipca_atual']*dolar['cpi']/last_cpi
dolar['paralelo_ipca_cpi_atual'] = dolar['paralelo_ipca_atual']*dolar['cpi']/last_cpi
dolar

In [ ]:
fig = plt.figure()
a1 = fig.add_axes([0,0,1,1])
a1.plot(dolar['comercial_igp_cpi_atual'], label = 'comercial (IGP-DI)')
a1.plot(dolar['paralelo_igp_cpi_atual'], label = 'paralelo (IGP-DI)')
a1.plot(dolar['comercial_ipca_cpi_atual'], label = 'comercial (IPCA)')
a1.plot(dolar['paralelo_ipca_cpi_atual'], label = 'paralelo (IPCA)')
a1.set_ylabel('US$')

plt.legend()
plt.show()

In [ ]:
dolar['comercial_igp_cpi_atual'].describe()

In [ ]:
dolar['comercial_ipca_cpi_atual'].describe()

Vamos apurar como a inflação se comportou ao longo desse tempo:

In [ ]:
dolar['pct_igp'] = 100*dolar['igp'].pct_change()
dolar['pct_ipca'] = 100*dolar['ipca'].pct_change()
dolar

In [ ]:
dolar['pct_igp'].describe()

In [ ]:
dolar['pct_ipca'].describe()

In [ ]:
fig = plt.figure()
a1 = fig.add_axes([0,0,1,1])
plt.title('Inflação')

a1.plot(dolar['pct_igp'], label = 'IGP-DI')
a1.plot(dolar['pct_ipca'], label = 'IPCA')
a1.set_ylabel('%')

plt.legend()
plt.show()

In [ ]:
fig = plt.figure()
a1 = fig.add_axes([0,0,1,1])
plt.title('Inflação pós Plano Real')

data_plano_real = '1994-07-01'
a1.plot(dolar[dolar.index > data_plano_real]['pct_igp'], label = 'IGP-DI')
a1.plot(dolar[dolar.index > data_plano_real]['pct_ipca'], label = 'IPCA')
a1.set_ylabel('%')

plt.legend()
plt.show()

In [ ]:
dolar[dolar.index > data_plano_real]['pct_igp'].describe()

In [ ]:
dolar[dolar.index > data_plano_real]['pct_ipca'].describe()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
plt.title('Taxa de câmbio (deflacionada pelo IGP-DI e CPI)')
ax.set_ylabel('Taxa de câmbio (R\$/US\$)')

ax.plot(dolar['paralelo_igp_cpi_atual'], color='red', label = 'paralelo')
ax.plot(dolar['comercial_igp_cpi_atual'], color='green', label = 'comercial')

threshold = 2
ax.fill_between(dolar.index, 0, 1, where=dolar['pct_igp'] > threshold,
                color='lightpink', alpha=0.7, transform=ax.get_xaxis_transform(),
                label='IGP-DI > {}%'.format(threshold))

plt.legend(loc = 'upper right')

plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
plt.title('Taxa de câmbio (deflacionada pelo IPCA e CPI)')
ax.set_ylabel('Taxa de câmbio (R\$/US\$)')

ax.plot(dolar['paralelo_ipca_cpi_atual'], color='red', label = 'paralelo')
ax.plot(dolar['comercial_ipca_cpi_atual'], color='green', label = 'comercial')

threshold = 2
ax.fill_between(dolar.index, 0, 1, where=dolar['pct_ipca'] > threshold,
                color='lightpink', alpha=0.7, transform=ax.get_xaxis_transform(),
                label='IPCA > {}%'.format(threshold))

plt.legend(loc = 'upper right')

plt.show()

In [ ]:
import numpy as np

fig, (ax1, ax2) = plt.subplots(2, sharex=True)
fig.suptitle('Taxa de câmbio')

threshold = 2

ax1.set_ylabel('R\$/US\$')
ax1.set_title('Deflacionada pelo IGP-DI e CPI')

ax1.plot(dolar['paralelo_igp_cpi_atual'], color='red', label = 'paralelo')
ax1.plot(dolar['comercial_igp_cpi_atual'], color='green', label = 'comercial')
ax1.axhline(dolar['paralelo_igp_cpi_atual'].mean(), linestyle='dotted', color='red', label = 'paralelo (média)')
ax1.axhline(dolar['comercial_igp_cpi_atual'].mean(), linestyle='dotted', color='green', label = 'comercial (média)')

ax1.fill_between(dolar.index, 0, 1, where=dolar['pct_igp'] > threshold,
                color='lightpink', alpha=0.7, transform=ax1.get_xaxis_transform(),
                label='IGP-DI > {}%'.format(threshold))

ax1.legend(bbox_to_anchor=(1.3, 0.6),fontsize=8)
ax1.set_yticks(np.arange(0, 1 + max(dolar['paralelo_igp_cpi_atual'].max(),dolar['comercial_igp_cpi_atual'].max()), 1))
                
ax2.set_ylabel('R\$/US\$')
ax2.set_title('Deflacionada pelo IPCA e CPI')

ax2.plot(dolar['paralelo_ipca_cpi_atual'], color='red', label = 'paralelo')
ax2.plot(dolar['comercial_ipca_cpi_atual'], color='green', label = 'comercial')
ax2.axhline(dolar['paralelo_ipca_cpi_atual'].mean(), linestyle='dotted', color='red', label = 'paralelo (média)')
ax2.axhline(dolar['comercial_ipca_cpi_atual'].mean(), linestyle='dotted', color='green', label = 'comercial (média)')

ax2.fill_between(dolar.index, 0, 1, where=dolar['pct_ipca'] > threshold,
                color='lightpink', alpha=0.7, transform=ax2.get_xaxis_transform(),
                label='IPCA > {}%'.format(threshold))

ax2.legend(bbox_to_anchor=(1.3, 0.6),fontsize=8)

plt.show()